# Walmart : predict weekly sales

# Introduction

## Import libraries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
import plotly.express as px
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV

## Import data

In [78]:
dataset = pd.read_csv("Walmart_Store_sales.csv")
dataset.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


# Part 1 : EDA and data preprocessing

Start your project by exploring your dataset : create figures, compute some statistics etc...


## 1. Basic Statistics

In [79]:
print("Number of rows : {}".format(dataset.shape[0]))
print()

print("Display of dataset: ")
display(dataset.head(10))
print()

print("Basics statistics: ")
data_desc = dataset.describe(include="all")
display(data_desc)
print()

print("Percentage of missing values: ")
display(100 * dataset.isnull().sum() / dataset.shape[0])


Number of rows : 150

Display of dataset: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092
5,4.0,28-05-2010,1857533.70,0.0,NaN,2.756,126.160226,7.896
6,15.0,03-06-2011,695396.19,0.0,69.80,4.069,134.855161,7.658
7,20.0,03-02-2012,2203523.20,0.0,39.93,3.617,213.023622,6.961
8,14.0,10-12-2010,2600519.26,0.0,30.54,3.109,NaN,NaN
9,3.0,NaN,418925.47,0.0,60.12,3.555,224.132020,6.833



Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

## 2. Visualizations

In [80]:
# Scatter plot of Weekly_Sales for each variable
for col in dataset.drop('Weekly_Sales', axis=1).columns:
    fig = px.scatter(dataset, x=col, y='Weekly_Sales',
               width=600, height=300,).update_layout(margin=dict(l=10, r=10, t=10, b=10))
    fig.show()
    img_name = f"img/scatter_WeeklySales_vs_{col}.png"
    # fig.write_image(img_name) # To save image

In [81]:
# Matrix
correlation_matrix = dataset.corr(numeric_only=True)
fig_corr = px.imshow(correlation_matrix,
                     text_auto=True,
                     aspect="auto",
                     title="Correlation matrix"
                    ).update_layout(margin=dict(l=10, r=10, t=50, b=10))

file_name_corr = "img/correlation_heatmap.png"
# fig_corr.write_image(file_name_corr)  # To save image

fig_corr.show()

## 3. Preprocessing with pandas

### 3.1. Cleaning data

In [82]:
# Drop lines where Weekly_Sales values are missing :
df_clean = dataset.dropna(subset=["Weekly_Sales"])

In [83]:
# Drop lines containing invalid values or outliers :
# outliers : all the numeric features that don't fall within the range : $[\bar{X} - 3\sigma, \bar{X} + 3\sigma]$.

colonnes_to_clean = ["Temperature", "Fuel_Price", "CPI", "Unemployment"]

# Cleaning function
def clean_outliers_3sigma(df, columns):
    """
    Drop lines containing invalid values or outliers :
    outliers : all the numeric features that don't fall within the range of 3 sigma rule
    """
    # Initialize a mask that initially keeps ALL rows (True)
    mask_to_keep = pd.Series(True, index=df.index)
    
    # Iteration over the columns
    for col in columns:
        # Calculation of statistics
        mean = df[col].mean()
        std = df[col].std()
        lower_bound = mean - 3 * std
        upper_bound = mean + 3 * std
        
        # Creation of the mask for the current column
        # Values between the bounds are kept
        current_col_mask = (df[col] >= lower_bound) & (df[col] <= upper_bound)
        
        # Combination of masks
        # A row must satisfy the rule in all previous columns AND the current one
        mask_to_keep = mask_to_keep & current_col_mask
    
    # Application of the mask
    return df[mask_to_keep]

df_clean = clean_outliers_3sigma(df_clean, colonnes_to_clean)

In [84]:
# Basic stats
print("Number of rows : {}".format(df_clean.shape[0]))
print()

print("Basics statistics: ")
data_desc = df_clean.describe(include="all")
display(data_desc)
print()

print("Percentage of missing values: ")
display(100 * df_clean.isnull().sum() / df_clean.shape[0])

Number of rows : 90

Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,90.000000,80,9.000000e+01,80.000000,90.00000,90.000000,90.000000,90.000000
unique,NaN,62,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,25-03-2011,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.900000,NaN,1.233865e+06,0.075000,61.06100,3.318444,179.524905,7.389733
std,6.204475,NaN,6.647250e+05,0.265053,17.74604,0.484399,39.554303,0.982729
min,1.000000,NaN,2.689290e+05,0.000000,18.79000,2.548000,126.128355,5.143000
25%,4.000000,NaN,5.617240e+05,0.000000,45.34250,2.814750,132.602339,6.642250
50%,9.000000,NaN,1.260826e+06,0.000000,61.45000,3.468000,197.166416,7.419000
75%,15.750000,NaN,1.807159e+06,0.000000,75.79250,3.737750,214.855374,8.099000



Percentage of missing values: 


Store            0.000000
Date            11.111111
Weekly_Sales     0.000000
Holiday_Flag    11.111111
Temperature      0.000000
Fuel_Price       0.000000
CPI              0.000000
Unemployment     0.000000
dtype: float64

### 3.2. Create usefull data

In [85]:
# Create usable features from the *Date* column :
# Create new columns that contain the following numeric features : year, month, day, day of week

df_clean["Date"] = pd.to_datetime(df_clean["Date"], format = "%d-%m-%Y")

df_clean['year'] = df_clean['Date'].dt.year
df_clean['month'] = df_clean['Date'].dt.month
df_clean['day'] = df_clean['Date'].dt.day
df_clean['weekday'] = df_clean['Date'].dt.dayofweek

df_clean = df_clean.drop("Date", axis=1)
df_clean.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day,weekday
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0
6,15.0,695396.19,0.0,69.80,4.069,134.855161,7.658,2011.0,6.0,3.0,4.0
7,20.0,2203523.20,0.0,39.93,3.617,213.023622,6.961,2012.0,2.0,3.0,4.0


In [86]:
# Scatter plot of Weekly_Sales for each new date variable
for col in ["year", "month", "day", "weekday"]:
    fig = px.scatter(df_clean, x=col, y='Weekly_Sales',
               width=600, height=300,).update_layout(margin=dict(l=10, r=10, t=10, b=10))
    fig.show()
    img_name = f"img/scatter_WeeklySales_vs_{col}.png"
    # fig.write_image(img_name) # To save image

## 4. Preprocessing for model

### 4.1. Separate target variable Y from features X

In [87]:
target_name = "Weekly_Sales"

print("Separating labels from features...")
Y = df_clean.loc[:, target_name]
X = df_clean.drop(target_name, axis=1)  # All columns are kept, except the target
print("...Done.")
print(Y.head())
print()
print(X.head())
print()

Separating labels from features...
...Done.
0    1572117.54
1    1807545.43
4    1644470.66
6     695396.19
7    2203523.20
Name: Weekly_Sales, dtype: float64

   Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
0    6.0           NaN        59.61       3.045  214.777523         6.858   
1   13.0           0.0        42.38       3.435  128.616064         7.470   
4    6.0           0.0        78.89       2.759  212.412888         7.092   
6   15.0           0.0        69.80       4.069  134.855161         7.658   
7   20.0           0.0        39.93       3.617  213.023622         6.961   

     year  month   day  weekday  
0  2011.0    2.0  18.0      4.0  
1  2011.0    3.0  25.0      4.0  
4  2010.0    5.0  28.0      4.0  
6  2011.0    6.0   3.0      4.0  
7  2012.0    2.0   3.0      4.0  



### 4.2. Split data in train-test set

In [88]:
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



### 4.3. Pipelines for preprocessing

In [89]:
# Pipeline for numeric features
numeric_features = ["Temperature","Fuel_Price","CPI","Unemployment", "year","month","day","weekday"]
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),  # missing values will be replaced by columns' median
        ("scaler", StandardScaler())
    ])

In [90]:
# Pipeline for categorical features
categorical_features = ["Store","Holiday_Flag"]
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),  # missing values will be replaced by most frequent value
        ("encoder", OneHotEncoder(drop="first"))  # first column will be dropped to avoid creating correlations between features
    ])

In [91]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

In [92]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print("...Done.")
print(X_train[0:5])
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test)
print("...Done.")
print(X_test[0:5, :])
print()

Performing preprocessings on train set...
     Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
127   16.0           0.0        61.79       2.711  189.523128         6.868   
63     5.0           0.0        69.17       3.594  224.019287         5.422   
35    19.0           0.0        33.26       3.789  133.958742         7.771   
10     8.0           0.0        82.92       3.554  219.070197         6.425   
95     1.0           0.0        74.78       2.854  210.337426         7.808   

       year  month   day  weekday  
127  2010.0    7.0   9.0      4.0  
63   2012.0   10.0  19.0      4.0  
35   2011.0    3.0  25.0      4.0  
10   2011.0    8.0  19.0      4.0  
95   2010.0    5.0  14.0      4.0  
...Done.
[[ 0.04260362 -1.26840641  0.20507788 -0.55534542 -1.1763434   0.147002
  -0.86859506  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          1.          0. 

# Part 2 : Baseline model (linear regression)

## 1. Train Baseline Model

In [93]:
model = LinearRegression()

print("Training model...")
model.fit(X_train, Y_train)
print("...Done.")

Training model...
...Done.


## 2. Predictions

In [94]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = model.predict(X_train)
print("...Done.")
print(Y_train_pred[0:5])
print()

Predictions on training set...
...Done.
[ 611364.67099396  370577.26212486 1275740.37137492  879179.76718068
 1536772.70829879]



In [95]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = model.predict(X_test)
print("...Done.")
print(Y_test_pred[0:5])
print()

Predictions on test set...
...Done.
[1569238.89010088  687168.56801076 1896822.31339263 1788054.17144371
  393226.322252  ]



## 3. Performance assessment

In [96]:
# Print R^2 scores
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

R2 score on training set :  0.9868321417045137
R2 score on test set :  0.9352216314000095


In [97]:
# Print MAE
print(f"Mean Absolute Error on training set : {mean_absolute_error(Y_train, Y_train_pred)}")
print(f"Mean Absolute Error on test set : {mean_absolute_error(Y_test, Y_test_pred)}")
print(f"Ratio MAE test / MAE train :{mean_absolute_error(Y_test, Y_test_pred) /mean_absolute_error(Y_train, Y_train_pred)}")

Mean Absolute Error on training set : 56136.01460814163
Mean Absolute Error on test set : 110916.01394672472
Ratio MAE test / MAE train :1.9758441122152288


**Model is overfitting.**

## 4. Features importance

In [98]:
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = pipeline.named_steps['encoder'].get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
# Create a pandas DataFrame
coefs = pd.DataFrame(index = column_names, data = model.coef_.transpose(), columns=["coefficients_without_regularization"])

# Update features names
coefs.index = coefs.index.str.replace('x0_', 'Store_')
coefs.index = coefs.index.str.replace('x1_', 'Holliday_Flag_')
# Compute abs() and sort values
feature_importance = abs(coefs).sort_values(by = 'coefficients_without_regularization', ascending=False)

# Plot coefficients
fig_coef = px.bar(feature_importance)
fig_coef.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )

file_name_coef = "img/Feature_importante.png"
# fig_coef.write_image(file_name_coef)  # To save image
fig_coef.show()


# Part 3 : Fight overfitting

## 1. GridSearch with Ridge & Lasso

### 1.1. Ridge

In [99]:
# Perform grid search
print("Grid search...")
regressor = Ridge()
# Grid of values to be tested
params = {
    'alpha': [0.5, 1, 2, 5, 8, 10, 12, 15]
}
best_ridge = GridSearchCV(regressor, param_grid = params, cv = 5) # cv : the number of folds to be used for CV
best_ridge.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters for Ridge : ", best_ridge.best_params_)
print("Best R2 score : ", best_ridge.best_score_)


Grid search...
...Done.
Best hyperparameters for Ridge :  {'alpha': 0.5}
Best R2 score :  0.8992001574033262


### 1.1. Lasso

In [100]:
# Perform grid search
print("Grid search...")
regressor = Lasso(max_iter=50000)
# Grid of values to be tested
params = {
    'alpha': [0.5, 1, 2, 5, 8, 10, 12, 15]
}
best_lasso = GridSearchCV(regressor, param_grid = params, cv = 5) # cv : the number of folds to be used for CV
best_lasso.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters for Lasso : ", best_lasso.best_params_)
print("Best R2 score : ", best_lasso.best_score_)


Grid search...
...Done.
Best hyperparameters for Lasso :  {'alpha': 12}
Best R2 score :  0.9542767596085259


## 2. Predictions

### 2.1. Ridge

In [101]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred_ridge = best_ridge.predict(X_train)
print("...Done.")
print(Y_train_pred_ridge[0:5])
print()

Predictions on training set...
...Done.
[ 763327.13758185  382774.33616975 1267186.63275544  938028.24093102
 1384689.94773037]



In [102]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred_ridge = best_ridge.predict(X_test)
print("...Done.")
print(Y_test_pred_ridge[0:5])
print()

Predictions on test set...
...Done.
[1533502.64201449  720965.9825302  1509880.71081111 1658555.60019836
  506319.60029212]



### 2.2. Lasso

In [103]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred_lasso = best_lasso.predict(X_train)
print("...Done.")
print(Y_train_pred_lasso[0:5])
print()

Predictions on training set...
...Done.
[ 608486.14741045  360375.6091709  1282342.22717136  880760.53761354
 1546501.9382564 ]



In [104]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred_lasso = best_lasso.predict(X_test)
print("...Done.")
print(Y_test_pred_lasso[0:5])
print()

Predictions on test set...
...Done.
[1567232.25385247  687330.08532408 1873496.40697084 1787638.29306106
  385906.81556055]



## 3. Performance assessment

In [105]:
# Print R^2 scores
print("RIDGE / R2 score on training set : ", best_ridge.score(X_train, Y_train))
print("RIDGE / R2 score on test set : ", best_ridge.score(X_test, Y_test))
print()
print("LASSO / R2 score on training set : ", best_lasso.score(X_train, Y_train))
print("LASSO / R2 score on test set : ", best_lasso.score(X_test, Y_test))


RIDGE / R2 score on training set :  0.9644803784465207
RIDGE / R2 score on test set :  0.8823288586112248

LASSO / R2 score on training set :  0.9867401109095157
LASSO / R2 score on test set :  0.9371787904688713


In [106]:
print(f"RIDGE / Mean Absolute Error on training set : {mean_absolute_error(Y_train, Y_train_pred_ridge)}")
print(f"RIDGE / Mean Absolute Error on test set : {mean_absolute_error(Y_test, Y_test_pred_ridge)}")
print(f"RIDGE / Ratio MAE test / MAE train :{mean_absolute_error(Y_test, Y_test_pred_ridge) /mean_absolute_error(Y_train, Y_train_pred_ridge)}")
print()
print(f"LASSO / Mean Absolute Error on training set : {mean_absolute_error(Y_train, Y_train_pred_lasso)}")
print(f"LASSO / Mean Absolute Error on test set : {mean_absolute_error(Y_test, Y_test_pred_lasso)}")
print(f"LASSO / Ratio MAE test / MAE train :{mean_absolute_error(Y_test, Y_test_pred_lasso) /mean_absolute_error(Y_train, Y_train_pred_lasso)}")

RIDGE / Mean Absolute Error on training set : 98610.07413712014
RIDGE / Mean Absolute Error on test set : 163207.49574305254
RIDGE / Ratio MAE test / MAE train :1.6550793331329192

LASSO / Mean Absolute Error on training set : 56314.105112408295
LASSO / Mean Absolute Error on test set : 110013.20842306278
LASSO / Ratio MAE test / MAE train :1.9535639997024898


**Ridge model is less overfitting.**

## 4. Features importance

In [107]:
# Extract features importances for Lasso & Ridge
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = pipeline.named_steps['encoder'].get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names

data_dict = {
    'Feature': column_names,
    'Best_Ridge': best_ridge.best_estimator_.coef_,
    'Best_Lasso': best_lasso.best_estimator_.coef_
            }

coefficients = pd.DataFrame(data = data_dict)

# Update features names
coefficients['Feature'] = coefficients['Feature'].apply(lambda x: str(x).replace('x0_', 'Store_'))
coefficients['Feature'] = coefficients['Feature'].apply(lambda x: str(x).replace('x1_', 'Holliday_Flag_'))

# Merge all data in 1 df
df_merged = pd.merge(
    left = coefficients,
    right = feature_importance,
    left_on='Feature',
    right_index=True,
    how='left'
)

df_merged = df_merged.set_index('Feature')
df_merged = abs(df_merged).sort_values(by = 'Best_Ridge', ascending=False)
display(df_merged)

,Best_Ridge,Best_Lasso,coefficients_without_regularization
Feature,,,
Store_7.0,858931.836214,7.283897e+05,6.198300e+05
Store_3.0,847506.585799,1.231362e+06,1.250987e+06
Store_5.0,797821.103130,1.227611e+06,1.227433e+06
Store_4.0,769276.034745,1.760171e+06,2.204173e+06
Store_15.0,668952.156918,1.780564e+05,5.892577e+05
Store_9.0,628572.711148,1.094990e+06,1.102517e+06
Store_20.0,625998.142961,5.568093e+05,5.924033e+05
Store_13.0,616526.095189,1.624579e+06,2.066814e+06
Store_16.0,553958.625778,6.992878e+05,5.775860e+05


In [108]:
# Plottinng all features coefficients
fig_all_coef = px.bar(df_merged)
fig_all_coef.update_layout(
    showlegend=True,
    barmode='group',
    title="Features' coefficients for different regularizations",
    xaxis_title="Features",
    yaxis_title="Coefficients",
    margin={'l': 120, 'r': 20, 't': 50, 'b': 20},
        legend=dict(x=1.0,y=1.0,xanchor='right',yanchor='top',
                    bgcolor='rgba(255, 255, 255, 0.7)',
                    bordercolor='Black',borderwidth=1)
)

file_name_all_coef = "img/Feature_importante_all_models.png"
#fig_all_coef.write_image(file_name_all_coef)  # To save image
fig_all_coef.show()